<a href="https://colab.research.google.com/github/vishant016/140_VISHANT/blob/main/LAB5/1_linear_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import Numpy & PyTorch
import numpy as np
import torch

A tensor is a number, vector, matrix or any n-dimensional array.

## Problem Statement

We'll create a model that predicts crop yeilds for apples (*target variable*) by looking at the average temperature, rainfall and humidity (*input variables or features*) in different regions. 

Here's the training data:

>Temp | Rain | Humidity | Prediction
>--- | --- | --- | ---
> 73 | 67 | 43 | 56
> 91 | 88 | 64 | 81
> 87 | 134 | 58 | 119
> 102 | 43 | 37 | 22
> 69 | 96 | 70 | 103

In a **linear regression** model, each target variable is estimated to be a weighted sum of the input variables, offset by some constant, known as a bias :

```
yeild_apple  = w11 * temp + w12 * rainfall + w13 * humidity + b1
```

It means that the yield of apples is a linear or planar function of the temperature, rainfall & humidity.



**Our objective**: Find a suitable set of *weights* and *biases* using the training data, to make accurate predictions.

## Training Data
The training data can be represented using 2 matrices (inputs and targets), each with one row per observation and one column for variable.

In [16]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

In [15]:
# Target (apples)
targets = np.array([[56], 
                    [81], 
                    [119], 
                    [22], 
                    [103]], dtype='float32')

Before we build a model, we need to convert inputs and targets to PyTorch tensors.

In [17]:
# Convert inputs and targets to tensors
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
targets

tensor([[ 56.],
        [ 81.],
        [119.],
        [ 22.],
        [103.]])

## Linear Regression Model (from scratch)

The *weights* and *biases* can also be represented as matrices, initialized with random values. The first row of `w` and the first element of `b` are use to predict the first target variable i.e. yield for apples, and similarly the second for oranges.

In [18]:
# Weights and biases
W = torch.rand((1,inputs.shape[1]),requires_grad=True)
print(W)

b = torch.zeros(size=(inputs.shape[0],1),requires_grad=True)
b


tensor([[0.9956, 0.1313, 0.1063]], requires_grad=True)


tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.]], requires_grad=True)

The *model* is simply a function that performs a matrix multiplication of the input `x` and the weights `w` (transposed) and adds the bias `b` (replicated for each observation).

$$
\hspace{2.5cm} X \hspace{1.1cm} \times \hspace{1.2cm} W^T \hspace{1.2cm}  + \hspace{1cm} b \hspace{2cm}
$$

$$
\left[ \begin{array}{cc}
73 & 67 & 43 \\
91 & 88 & 64 \\
\vdots & \vdots & \vdots \\
69 & 96 & 70
\end{array} \right]
%
\times
%
\left[ \begin{array}{cc}
w_{11} & w_{21} \\
w_{12} & w_{22} \\
w_{13} & w_{23}
\end{array} \right]
%
+
%
\left[ \begin{array}{cc}
b_{1} & b_{2} \\
b_{1} & b_{2} \\
\vdots & \vdots \\
b_{1} & b_{2} \\
\end{array} \right]
$$

In [19]:
# Define the model
predict = lambda X,W,b: X@W.T+b


The matrix obtained by passing the input data to the model is a set of predictions for the target variables.

In [20]:
# Generate predictions
y_h = predict(inputs,W,b)
y_h

tensor([[ 86.0538],
        [108.9661],
        [110.3878],
        [111.1375],
        [ 88.7505]], grad_fn=<AddBackward0>)

In [21]:
# Compare with targets
targets


tensor([[ 56.],
        [ 81.],
        [119.],
        [ 22.],
        [103.]])

Because we've started with random weights and biases, the model does not perform a good job of predicting the target varaibles.

## Loss Function

We can compare the predictions with the actual targets, using the following method: 
* Calculate the difference between the two matrices (`preds` and `targets`).
* Square all elements of the difference matrix to remove negative values.
* Calculate the average of the elements in the resulting matrix.

The result is a single number, known as the **mean squared error** (MSE).

In [22]:
# MSE loss
loss_fn = torch.nn.MSELoss()
my_loss = lambda Y,Yhat : ((Y-Yhat)**2).mean()


In [24]:
# Compute loss
loss = loss_fn(targets,y_h)
loss


tensor(1981.6080, grad_fn=<MseLossBackward>)

The resulting number is called the **loss**, because it indicates how bad the model is at predicting the target variables. Lower the loss, better the model. 

## Compute Gradients

With PyTorch, we can automatically compute the gradient or derivative of the `loss` w.r.t. to the weights and biases, because they have `requires_grad` set to `True`.

More on autograd:  https://pytorch.org/docs/stable/autograd.html#module-torch.autograd

In [25]:


# Compute gradients
loss.backward()




The gradients are stored in the `.grad` property of the respective tensors.

In [26]:
# Gradients for weights
W.grad

tensor([[4839.3530, 2314.2161, 1953.3014]])

In [27]:
# Gradients for bias
b.grad

tensor([[12.0215],
        [11.1865],
        [-3.4449],
        [35.6550],
        [-5.6998]])

A key insight from calculus is that the gradient indicates the rate of change of the loss, or the slope of the loss function w.r.t. the weights and biases. 

* If a gradient element is **postive**, 
    * **increasing** the element's value slightly will **increase** the loss.
    * **decreasing** the element's value slightly will **decrease** the loss.




* If a gradient element is **negative**,
    * **increasing** the element's value slightly will **decrease** the loss.
    * **decreasing** the element's value slightly will **increase** the loss.
    


The increase or decrease is proportional to the value of the gradient.

Finally, we'll reset the gradients to zero before moving forward, because PyTorch accumulates gradients.

In [28]:


W.grad.zero_(),b.grad.zero_()



(tensor([[0., 0., 0.]]), tensor([[0.],
         [0.],
         [0.],
         [0.],
         [0.]]))

## Adjust weights and biases using gradient descent

We'll reduce the loss and improve our model using the gradient descent algorithm, which has the following steps:

1. Generate predictions
2. Calculate the loss
3. Compute gradients w.r.t the weights and biases
4. Adjust the weights by subtracting a small quantity proportional to the gradient
5. Reset the gradients to zero

In [30]:
# Generate predictions
X = torch.from_numpy( np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32'))
Y = torch.from_numpy( np.array([[56], 
                    [81], 
                    [119], 
                    [22], 
                    [103]], dtype='float32'))

W = torch.rand((1,X.shape[1]),requires_grad=True)
b = torch.tensor([[0.0]],requires_grad=True)

predict = lambda X,W,b : X@W.T + b

W,b,W.grad,b.grad
predictions = predict(X,W,b)
predictions

tensor([[122.1994],
        [157.4492],
        [180.6913],
        [134.1945],
        [142.3553]], grad_fn=<AddBackward0>)

In [31]:
# Calculate the loss
loss = loss_fn(Y,predictions)
loss

tensor(5633.8218, grad_fn=<MseLossBackward>)

In [32]:
# Compute gradients
loss.backward()
print(W.grad,b.grad)

tensor([[12526.3740, 11212.7998,  7289.3955]]) tensor([[142.3559]])


In [34]:
# Adjust weights & reset gradients
alpha = 1e-7
with torch.no_grad():
    W -= alpha*(W.grad)
    b -= alpha*(b.grad)
W.grad.zero_(),b.grad.zero_()
W,b
predictions = predict(X,W,b)
predictions

tensor([[122.0015],
        [157.1899],
        [180.3898],
        [133.9915],
        [142.1102]], grad_fn=<AddBackward0>)

In [36]:
#print(w)
W

tensor([[0.9715, 0.6181, 0.2248]], requires_grad=True)

With the new weights and biases, the model should have a lower loss.

In [37]:
# Calculate loss

loss = loss_fn(Y,predictions)
loss

tensor(5600.3018, grad_fn=<MseLossBackward>)

## Train for multiple epochs

To reduce the loss further, we repeat the process of adjusting the weights and biases using the gradients multiple times. Each iteration is called an epoch.

In [38]:


# Train for 100 epochs
losses=[]
for i in range(100):
    yhat = predict(X,W,b)
    loss = loss_fn(Y,yhat)
    loss.backward()
    with torch.no_grad():
        W -= alpha*W.grad
        b -= alpha*b.grad
    losses.append(loss)
    W.grad.zero_(),b.grad.zero_()



In [39]:
# Calculate loss
loss_fn(Y,yhat)

tensor(3226.8379, grad_fn=<MseLossBackward>)

In [40]:
# Print predictions
predict(X,W,b)

tensor([[105.3815],
        [135.4256],
        [155.1604],
        [116.8572],
        [121.5927]], grad_fn=<AddBackward0>)

In [41]:
# Print targets
Y

tensor([[ 56.],
        [ 81.],
        [119.],
        [ 22.],
        [103.]])